# BROADCAST
> executor가 필요할 때마다 필요한 데이터를 줌

> 만약 데이터가 너무 크다면? 한 번만 보내놓고 해당 executor가 보관함

> 필요할 때마다 .value()로 broadcast 안에 있는 값을 씀

In [44]:
def load_movie_names():
    movie_dict = dict()
    
    with open('../dataset/ml-100k/u.item',encoding='ISO-8859-1') as f:
        for line in f:
            tok = line.split('|')[:2]
            movie_dict[int(tok[0])] = tok[1]
            
    return movie_dict

In [59]:
from pyspark import SparkConf, SparkContext
import collections

conf = SparkConf().setMaster("local").setAppName("RatingsHistogram")
sc = SparkContext.getOrCreate(conf=conf)

name_dict = sc.broadcast(load_movie_names())
# 얘를 필요한 executor에게 보내는 방식

lines = sc.textFile("file:/home/ej/github/codingplace/spark/dataset/ml-100k/u.data")
tok = lines.map(lambda x: (x.split()[1], 1))

In [53]:
name_dict

In [46]:
total = tok.reduceByKey(lambda x, y: x+y)

In [47]:
switch = total.map(lambda x: (x[1], x[0]))
switch = switch.sortByKey()
switch = switch.map(lambda x: (x[1], x[0]))

In [48]:
switch.collect()[::-1][0]

('50', 583)

In [55]:
movie_with_name = switch.map(lambda x: (name_dict.value[int(x[0])] ,x[1]))

In [58]:
movie_with_name.collect()[::-1][:5]

[('Star Wars (1977)', 583),
 ('Contact (1997)', 509),
 ('Fargo (1996)', 508),
 ('Return of the Jedi (1983)', 507),
 ('Liar Liar (1997)', 485)]